## Installing packages

TODO:


*   Check for the number problem again
*   Write a code to get random 1000 samples ( currently it is taking first 100)
*   Clean up code. Remove redundancieas, write imports properly
*   To increase run time, instead of repeatedly downloading the data set we can locally save it and just load it.
*   Similarly also maay load the output locally
*   Evaluation metric
      *    Bleu metric and meteor

when i tried this thing in english it is adding random number. Maybe it does the same in french?
* The reopening of the legislative period - english of the french
* Resumption of the session - english of the german


In [12]:
%pip install datasets transformers scikit-learn accelerate ipywidgets evaluate
!TMPDIR=./tmp pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/cu126

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Making imports

In [13]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

import evaluate

## Importing and pre-processing the dataset
TODO: change to 1000 later

In [41]:
# Load the Europarl dataset
dataset = load_dataset("europarl_bilingual", "de-fr")

# Inspect the dataset
print(dataset)
required_subset_dataset = dataset['train'].shuffle(seed=42).select(range(20))
df = pd.DataFrame(required_subset_dataset)


#Making a 80-20 Split of the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1942666
    })
})
Training set size: 16
Testing set size: 4


In [4]:
# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA available: True
GPU: NVIDIA GeForce GTX 1650 Ti


## now we look into importing the model

In [5]:
model_name = "Qwen/Qwen2-1.5B"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Define the function for translation

In [6]:
def translate_to_french(input_sentence):
    # Construct the prompt for translation
    prompt = f"Translate the following text from German into French: \"{input_sentence}\".\nOutput (French):"

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate the output
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,  # Limit the length of the generated output
            do_sample=False,     # Disable sampling to ensure deterministic output
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the output tokens to text
    translated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the translated part from the model's output
    # Assuming the model appends the translation after "Translation: "
    start_index = translated_text.find('Output (French): "') + len('Output (French): "')
    end_index = translated_text.find('"', start_index)
    translated_text = translated_text[start_index:].strip()

    translated_text = translated_text.split("\n")[0].strip()
    translated_text = translated_text.strip('".')
    return translated_text

In [7]:
#Example usage
input_sentence = required_subset_dataset[0]["translation"]["de"]
translation = translate_to_french(input_sentence)
print(f"{translation}")

L'introduction du droit d'occupation permanente est un exemple concret


In [8]:
required_subset_dataset[0]["translation"]["fr"]

"Le droit de séjour permanent, qui est à l'étude, en constitue un exemple concret."

In [9]:
def translate_test_set(test_data):
    translations = []
    for i, row in test_data.iterrows():
          german_sentence = row['translation']['de']
          french = translate_to_french(german_sentence)
          translations.append({
              'original_de': german_sentence,
              'translation': french,
          })
          print(f"Translated {i + 1}/{len(test_data)}: {german_sentence} -> {french}")
    return translations


TODO: Change this to 1k later

In [42]:
translated_results = translate_test_set(train_df)


# Save the results to a file
translated_df = pd.DataFrame(translated_results)
translated_df.to_csv("translated_test_set.csv", index=False, encoding="utf-8")

print("Translation completed and saved to 'translated_test_set.csv'.")

Translated 9/16: Der Entschließungsantrag von Joseph Daul ist nach langen Diskussionen und vielen Änderungsanträgen als Kompromiss quer durch alle Fraktionen im Agrarausschuss abgestimmt worden. -> L'avis de Joseph Daul a été adopté après de longues discussions et de nombreux amendements, par un accord entre toutes les groupes dans l'Assemblée générale des affaires agricoles
Translated 6/16: Sollen Transplantationen weiterentwickelt werden, muss ein starkes Bewusstsein der Menschen für die Organspende gefördert werden, und dies erfordert ein besonderes soziales Umfeld. -> Si les transplantations sont développées, il faut un grand esprit de la part des gens pour la don de l'organe, et cela nécessite un environnement social spécial
Translated 12/16: Im Zusammenhang mit ihrem hervorragenden Bericht hat sie richtigerweise betont, dass hier die Interessen der Kinder im Mittelpunkt stehen müssen. -> Dans le cadre de son excellent rapport, elle a correctement souligné que les intérêts des enf

In [47]:
references = []
for item in train_df['translation']:
    references.append([item['de'], item['fr']])

In [48]:
predictions = translated_df["translation"].to_numpy()


In [54]:
predictions[1]

"Si les transplantations sont développées, il faut un grand esprit de la part des gens pour la don de l'organe, et cela nécessite un environnement social spécial"

## Loading the BLEU Testing metric

In [49]:
bleu = evaluate.load("bleu")

In [50]:
results = bleu.compute(predictions=predictions, references=references)

In [51]:
print(results)

{'bleu': 0.17719535353088217, 'precisions': [0.4764705882352941, 0.23076923076923078, 0.12552301255230125, 0.07142857142857142], 'brevity_penalty': 1.0, 'length_ratio': 1.0714285714285714, 'translation_length': 510, 'reference_length': 476}
